In [1]:
from tslearn.utils import to_time_series_dataset
import json
import pandas as pd
import os

/home/woody/iwso/iwso092h/.venv/lib64/python3.11/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
label_df = pd.read_csv('labels.csv')

In [3]:
parent_dir = '/home/vault/empkins/tpD/D02/processed_data/facemo_data'

In [4]:
ids = os.listdir(parent_dir)

In [5]:
ids = os.listdir(parent_dir)

numeric_ids = [id for id in ids if id.isdigit()]

In [6]:
valid_ids = []

# List to store DataFrames from json files
coping_list = []

# Iterate through all IDs in the directory
ids = os.listdir(parent_dir)
for id in ids:
    tagret_path = os.path.join(parent_dir, str(id), 'latency.json')
    if os.path.isfile(tagret_path):  # Check if the file exists
        try:
            with open(tagret_path, 'r') as f:
                coping_data = json.load(f)
            df = pd.DataFrame.from_dict(coping_data)
            if not df.empty:
                coping_list.append(df)
                valid_ids.append(int(id))  # Add valid ID to the list
        except (FileNotFoundError, ValueError, json.JSONDecodeError):
            pass

id_df = pd.DataFrame({'Participant ID': valid_ids})

# combined_df = pd.read_excel('path_to_combined_df.xlsx') # If loaded from file

label_df['Participant ID'] = label_df['Participant ID'].astype(int)

reordered_df = id_df.merge(label_df, on='Participant ID', how='left')

In [7]:
reordered_df.shape

(100, 2)

In [8]:
len(coping_list)

100

In [9]:
coping_list = [df for df in coping_list if len(df) > 0]

In [10]:
X = to_time_series_dataset(coping_list)

In [11]:
y = reordered_df['Label']
y = pd.Series(y)

In [12]:
if y.isna().sum() > 0:
    print(f"Found {y.isna().sum()} NaN values in the labels. Handling them...")
    y.fillna(y.mode()[0], inplace=True)

Found 3 NaN values in the labels. Handling them...


In [13]:
y = y.apply(lambda x: 0 if x == 0.5 else x).astype(int)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [16]:
# from tslearn.neighbors import KNeighborsTimeSeriesClassifier
# knn = KNeighborsTimeSeriesClassifier(n_neighbors=4)
# knn.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(n_neighbors=4)

In [18]:
from tslearn.svm import TimeSeriesSVC
clf = TimeSeriesSVC(C=1.0, kernel="gak")
clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [17]:
y_pred = clf.predict(X_test)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)